In [1]:
# Importing necessary libraries
!pip install nltk
import numpy as np
import pandas as pd
import re
import nltk
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
from nltk.corpus import stopwords

In [2]:


#Reading the dataset
train = pd.read_csv(r'C:\Users\User-1\Downloads\Amazon ML Challenge\train.csv')
test = pd.read_csv(r'C:\Users\User-1\Downloads\Amazon ML Challenge\test.csv')

# To see first 5 rows of dataset

train.head()

,PRODUCT_ID,TITLE,BULLET_POINTS,DESCRIPTION,PRODUCT_TYPE_ID,PRODUCT_LENGTH
0,1925202,ArtzFolio Tulip Flowers Blackout Curtain for D...,[LUXURIOUS & APPEALING: Beautiful custom-made ...,NaN,1650,2125.980000
1,2673191,Marks & Spencer Girls' Pyjama Sets T86_2561C_N...,"[Harry Potter Hedwig Pyjamas (6-16 Yrs),100% c...",NaN,2755,393.700000
2,2765088,PRIKNIK Horn Red Electric Air Horn Compressor ...,"[Loud Dual Tone Trumpet Horn, Compatible With ...","Specifications: Color: Red, Material: Aluminiu...",7537,748.031495
3,1594019,ALISHAH Women's Cotton Ankle Length Leggings C...,[Made By 95%cotton and 5% Lycra which gives yo...,AISHAH Women's Lycra Cotton Ankel Leggings. Br...,2996,787.401574
4,283658,The United Empire Loyalists: A Chronicle of th...,NaN,NaN,6112,598.424000


In [3]:
train = train[['TITLE' ,'BULLET_POINTS', 'PRODUCT_LENGTH']]

In [4]:
train = train.fillna('@')

In [5]:
train['x'] = train['TITLE'] + train['BULLET_POINTS']

In [6]:
test.isnull().sum()

PRODUCT_ID              0
TITLE                   5
BULLET_POINTS      275922
DESCRIPTION        380001
PRODUCT_TYPE_ID         0
dtype: int64

In [7]:
test = test[['TITLE' ,'BULLET_POINTS']]
test = test.fillna('A')
test['x'] = test['TITLE'] + test['BULLET_POINTS']

In [8]:
train['x'] = train['x'].apply(lambda x : re.sub('[^a-zA-Z#\s]*','',x))
test['x'] = test['x'].apply(lambda x : re.sub('[^a-zA-Z#\s]*','',x))

### 3. Removing Short Words

We have to be a little careful here in selecting the length of the words which we want to remove. So, I have decided to remove all the words having length 3 or less. For example, terms like “hmm”, “oh” are of very little use. It is better to get rid of them.

In [9]:
train['x'] = train['x'].apply(lambda x : ' '.join([w for w in x.split() if len(w)>3]))

In [10]:
test['x'] = test['x'].apply(lambda x : ' '.join([w for w in x.split() if len(w)>3]))

### 4. Text Normalization

Here we will use nltk’s PorterStemmer() function to normalize the tweets. But before that we will have to tokenize the tweets. Tokens are individual terms or words, and tokenization is the process of splitting a string of text into tokens.

Above given code takes too much time to run and as an data analyst we cant be happy with this lines of code since we are going to work with very huge dataset aso i have wriiten alternative code for this task

In [11]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer() 

In [12]:
train['x'] = train['x'].apply(lambda x : x.split())

In [13]:
train['x'] = train['x'].apply(lambda y :  [lemmatizer.lemmatize(w) for w in y])

In [14]:
test['x'] = test['x'].apply(lambda x : x.split())
test['x'] = test['x'].apply(lambda y :  [lemmatizer.lemmatize(w) for w in y])

Now lets sttitch this token back to retreive our cleaned tweet


In [15]:
train['x'] = train['x'].apply(lambda y : ' '.join(w for w in y))
    

In [16]:
test['x'] = test['x'].apply(lambda y : ' '.join(w for w in y))

### Removing stop words

In [17]:
print(set(stopwords.words('english')))

{'doing', 'those', 'above', 'mustn', 'did', 'when', 'both', 'about', "couldn't", 'he', "she's", 'just', 'each', 'aren', 'ain', 'they', 'than', 'himself', 'hadn', 'i', 'to', 'won', "wouldn't", 'his', 'or', 'any', "aren't", 'hers', 'most', 're', 'can', 'him', 'were', 'only', 'ourselves', 'very', 'why', 'yourself', 'down', 'here', 'for', 'nor', 'below', 'o', 'hasn', "mustn't", 'not', 'which', 'same', 'between', 'how', 'but', "didn't", 'our', 'the', 'couldn', 'into', 'further', "isn't", 'their', 'itself', 'them', 've', 'over', 'me', 'was', "needn't", 'at', 'there', 'she', 'once', "should've", "hasn't", 'with', 'does', 'y', 'up', 'my', 'in', 'needn', 'm', 'where', "you've", 'shouldn', 'from', 'its', 'wouldn', 'such', 'other', "haven't", "you're", 'through', 'have', 'it', "you'll", 'that', 'haven', 'out', 'because', 'yours', 's', "weren't", 'so', "it's", 'doesn', 'on', 'then', 'a', 'all', 'more', 'no', 'being', 'don', 'ma', 'am', 'during', 'too', 'you', "don't", "won't", 'we', 'theirs', 'hav

In [18]:
stop_words = set(stopwords.words('english'))

In [19]:
train['x'] = train['x'].apply(lambda x : ' '.join([w for w in x.split() if not w in stop_words]))

In [20]:
test['x'] = test['x'].apply(lambda x : ' '.join([w for w in x.split() if not w in stop_words]))

In [ ]:
train.isnull().sum()

### 2) Tfidf Vectorizer

In [21]:
from sklearn.feature_extraction.text import CountVectorizer , TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(max_df = 0.9 , min_df = 2 , stop_words = 'english' , max_features = 1000)
df_train = tfidf_vectorizer.fit_transform(train['x'])

In [22]:
df_test = tfidf_vectorizer.fit_transform(test['x'])

In [23]:
train.to_csv('amazon_train.csv')
test.to_csv('amazon_test.csv')

In [24]:
np.shape(df_train)


(2249698, 1000)

In [25]:
y_train = train[['PRODUCT_LENGTH']]

## Modelling

In [26]:
from sklearn.tree import DecisionTreeRegressor
dtr = DecisionTreeRegressor()

In [ ]:
dtr.fit(df_train , y_train)
pred = dtr.predict(df_test)
# predicted = dtr.predict(df_train)
pred = pd.DataFrame(pred)

In [ ]:
from sklearn import metrics
(1-metrics.mean_absolute_percentage_error(y_train,predicted))

In [ ]:
pred.to_csv(r'C:\Users\User-1\Downloads\Amazon ML Challenge\submit.csv')

# METHOD - 2 Using LSTM

In [34]:
from keras.preprocessing.text import one_hot


onehot_repr=[one_hot(words,5000)for words in df['nice_tweet']] 


In [35]:
from keras.preprocessing.sequence import pad_sequences
embedded_doc = pad_sequences(onehot_repr , padding = 'pre' , maxlen = 20)

In [36]:
from keras.models import Sequential
from keras.layers import Dense , Dropout , LSTM , Bidirectional , Embedding

model=Sequential()
model.add(Embedding(5000 , 50 ,input_length=20))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 20, 50)            250000    
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               60400     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 101       
Total params: 310,501
Trainable params: 310,501
Non-trainable params: 0
_________________________________________________________________
None


In [37]:
X_lstm = np.array(embedded_doc)
y_lstm = df['label']
model.fit(X_lstm , y_lstm)

999/999 [==============================] - 25s 22ms/step - loss: 0.2252 - accuracy: 0.9343


In [38]:
a = model.predict_classes(X_lstm)

C:\Users\User-1\Anaconda3\lib\site-packages\tensorflow\python\keras\engine\sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [39]:
len(a)

31962

 ### So while using lstm training accuracy is around 95 % whereas while using concepts of nlp like vectorizer accuracy is just 50 %

In [40]:
np.shape(embedded_doc)

(31962, 20)